## Preprocessing

In [2]:
from openfe import OpenFE, tree_to_formula, transform, TwoStageFeatureSelector
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('F:/Kỳ 4/Thầy Hùng/OpenFE/data/BoT-IoT/All features/bot-iot.csv')

C:\Users\chaut\AppData\Local\Temp\ipykernel_26080\326756477.py:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('F:/Kỳ 4/Thầy Hùng/OpenFE/data/BoT-IoT/All features/bot-iot.csv')


In [4]:
df.head(5)

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.12704,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.20000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.12704,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.12704,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.12704,96,75,1.135260,1.129970,770,602,1,DoS,HTTP


In [5]:
df.shape

(3668522, 46)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3668522 entries, 0 to 3668521
Data columns (total 46 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   pkSeqID                           int64  
 1   stime                             float64
 2   flgs                              object 
 3   flgs_number                       int64  
 4   proto                             object 
 5   proto_number                      int64  
 6   saddr                             object 
 7   sport                             object 
 8   daddr                             object 
 9   dport                             object 
 10  pkts                              int64  
 11  bytes                             int64  
 12  state                             object 
 13  state_number                      int64  
 14  ltime                             float64
 15  seq                               int64  
 16  dur                               fl

In [7]:
df['attack'].unique()

array([1, 0], dtype=int64)

In [8]:
df['attack'].value_counts()

attack
1    3668045
0        477
Name: count, dtype: int64

In [9]:
df.select_dtypes(include=['object']).columns

Index(['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state',
       'category', 'subcategory'],
      dtype='object')

In [10]:
columns_to_convert = ['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state',
       'category', 'subcategory']

In [11]:
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [12]:
df.dropna()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory


In [13]:
df.isnull().sum()

pkSeqID                                   0
stime                                     0
flgs                                3668522
flgs_number                               0
proto                               3668522
proto_number                              0
saddr                               3668522
sport                                  9052
daddr                               3668522
dport                                  9052
pkts                                      0
bytes                                     0
state                               3668522
state_number                              0
ltime                                     0
seq                                       0
dur                                       0
mean                                      0
stddev                                    0
sum                                       0
min                                       0
max                                       0
spkts                           

In [14]:
#Encode
label_encoder = LabelEncoder()
cols_to_encode = ['flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'state', 'category', 'subcategory']
for col in cols_to_encode:
    if df[col].dtype == 'object':
        df[col] = label_encoder.fit_transform(df[col])

In [15]:
for col in df.columns:
  if len(df[col].unique()) <= 1:
    print(col)
    df.drop(col, axis=1, inplace=True)

flgs
proto
saddr
daddr
state
category
subcategory


In [17]:
features = df.drop(['attack'], axis=1)
labels = df[['attack']]

In [18]:
features.head()

,pkSeqID,stime,flgs_number,proto_number,sport,dport,pkts,bytes,state_number,ltime,...,TnP_PerProto,TnP_Per_Dport,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP
0,1,1.528089e+09,1,1,49960.0,80.0,8,1980,1,1.528089e+09,...,772,772,1.142970,1.12704,96,75,1.133720,1.129970,770,602
1,2,1.528089e+09,1,2,-1.0,-1.0,2,120,2,1.528089e+09,...,8,8,0.003857,15267.20000,1,2,0.005142,0.005142,2,6
2,3,1.528089e+09,1,1,49962.0,80.0,8,2126,1,1.528089e+09,...,772,772,1.142970,1.12704,96,75,1.135100,1.129970,770,602
3,4,1.528089e+09,1,1,49964.0,80.0,8,2024,1,1.528089e+09,...,772,772,1.142970,1.12704,96,75,1.135140,1.129970,770,602
4,5,1.528089e+09,1,1,49966.0,80.0,8,2319,1,1.528089e+09,...,772,772,1.142970,1.12704,96,75,1.135260,1.129970,770,602


In [19]:
df['attack'].value_counts()

attack
1    3668045
0        477
Name: count, dtype: int64

## Training

In [20]:
X, _, y, _ = train_test_split(features, labels, train_size=500000, stratify=labels, random_state=42)

In [21]:
y.value_counts()

attack
1         499935
0             65
Name: count, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
y.value_counts()

attack
1         499935
0             65
Name: count, dtype: int64

## Generate new feature by OpenFE

In [ ]:
ofe = OpenFE()
features = ofe.fit(data = X_train, label = y_train) 

The number of candidate features is 7530
Start stage I selection.


  0%|          | 0/4 [00:00<?, ?it/s]

: 

In [ ]:
# Set hyperparameters for binary classification
params = {
    'objective': 'binary',  # Đặt mục tiêu là phân loại nhị phân
    'metric': 'binary_error',  # Đánh giá bằng binary log loss
    'boosting_type': 'gbdt',
    'seed' : 1
}

## Score function

In [ ]:
def get_score(train_x, test_x, train_y, test_y):
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
    gbm = lgb.LGBMClassifier(**params)
    gbm.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    pred = gbm.predict(test_x)
    score = accuracy_score(test_y, pred)
    return score


### Before Feature Generation

In [ ]:
# get baseline score
score = get_score(X_train, X_test, y_train, y_test)
print("The score before feature generation is", score)

[LightGBM] [Info] Number of positive: 1437, number of negative: 4563
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000467 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2713
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239500 -> initscore=-1.155423
[LightGBM] [Info] Start training from score -1.155423
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

### After Feature Generation

In [ ]:
X_train, X_test = transform(X_train, X_test, ofe.new_features_list[:10], n_jobs = 1) 

In [ ]:
X_train.columns

Index(['ip_src', 'ip_dst', 'prt_src', 'prt_dst', 'proto', 'num_pkts',
       'mean_iat', 'std_iat', 'min_iat', 'max_iat', 'mean_pkt_len',
       'num_bytes', 'num_psh_flags', 'num_rst_flags', 'num_urg_flags',
       'std_pkt_len', 'min_pkt_len', 'max_pkt_len', 'autoFE_f_0', 'autoFE_f_1',
       'autoFE_f_2', 'autoFE_f_3', 'autoFE_f_4', 'autoFE_f_5', 'autoFE_f_6',
       'autoFE_f_7', 'autoFE_f_8', 'autoFE_f_9'],
      dtype='object')

In [ ]:
score = get_score(X_train, X_test, y_train, y_test)

[LightGBM] [Info] Number of positive: 1437, number of negative: 4563
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000708 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3514
[LightGBM] [Info] Number of data points in the train set: 6000, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239500 -> initscore=-1.155423
[LightGBM] [Info] Start training from score -1.155423
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [ ]:
# OpenFE recommends a list of new features. We include the top 10
# generated features to see how they influence the model performance
print("The score after feature generation is", score)
print("The top 10 generated features are")
for feature in ofe.new_features_list[:10]:
    print(tree_to_formula(feature))

The score after feature generation is 1.0
The top 10 generated features are
(num_pkts-min_pkt_len)
(std_iat-min_pkt_len)
GroupByThenRank(proto,ip_dst)
CombineThenFreq(ip_src,min_pkt_len)
GroupByThenMedian(max_pkt_len,ip_src)
GroupByThenMin(max_pkt_len,ip_src)
GroupByThenRank(min_pkt_len,ip_dst)
GroupByThenRank(min_pkt_len,ip_src)
Combine(num_rst_flags,min_pkt_len)
GroupByThenMin(prt_src,ip_src)
